In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import time

# モデルディレクトリ
base_model_name = "llm-jp/llm-jp-3-3.7b"

# int4量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16  # 計算をfloat16で実行
)

# トークナイザとモデルのロード
tokenizer_base = AutoTokenizer.from_pretrained(base_model_name)
model_base = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float32
)

model_base_quantized = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# モデルのデバイスを表示
def print_model_device(model, model_name="Model"):
    """
    モデルのデバイスを表示する関数
    """
    device_count = {}
    for name, param in model.named_parameters():
        device = param.device
        if device in device_count:
            device_count[device] += param.numel()
        else:
            device_count[device] = param.numel()
    
    print(f"{model_name} is distributed across:")
    for device, count in device_count.items():
        print(f"  {device}: {count} parameters")
    print()

# Base model のデバイス情報を表示
print_model_device(model_base, model_name="Base Model")

# Quantized model のデバイス情報を表示
print_model_device(model_base_quantized, model_name="Quantized Model")


Base Model is distributed across:
  cuda:0: 3782913024 parameters

Quantized Model is distributed across:
  cuda:0: 2197466112 parameters



In [3]:
# JGLUEデータセットのロード
jglue_dataset = load_dataset("shunk031/JGLUE", "JNLI", cache_dir="./datasets_cache")



# データセットを分割
train_data = jglue_dataset["train"]
validation_data = jglue_dataset["validation"]

# テストデータをデータフレームに変換
train_data_df = pd.DataFrame(train_data)

In [4]:
def evaluate_and_record(model, tokenizer, dataset, model_name="Model"):
    """
    モデルを評価し、正解・不正解を記録する。
    """
    results = []
    start_time = time.time()  # 実行時間計測開始
    for idx, sample in enumerate(tqdm(dataset, desc=f"Evaluating {model_name}")):
        input_text = sample["sentence1"] + " " + sample.get("sentence2", "")  # NLIタスク用
        label = str(sample["label"])  # ラベルを文字列化（モデルの出力と比較するため）

        # モデルで出力生成
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

        # 不要なフィールドを削除
        if "token_type_ids" in inputs:
            del inputs["token_type_ids"]

        # 生成
        outputs = model.generate(**inputs, max_new_tokens=10, pad_token_id=tokenizer.pad_token_id)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # 予測結果を記録
        results.append({
            "id": idx,
            "input": input_text,
            "label": label,
            "prediction": prediction
        })

    # 実行時間計測終了
    end_time = time.time()
    print(f"{model_name} evaluation completed in {end_time - start_time:.2f} seconds.")

    return pd.DataFrame(results)

In [5]:
# Base model の評価
print("Evaluating Base Model...")
base_results = evaluate_and_record(model_base, tokenizer_base, train_data, model_name="Base Model")

# Quantized model の評価
print("Evaluating Quantized Model...")
quantized_results = evaluate_and_record(model_base_quantized, tokenizer_base, train_data, model_name="Quantized Model")

# 正解ラベルとの一致を判定
base_results["is_correct_base"] = base_results["prediction"] == base_results["label"]
quantized_results["is_correct_quantized"] = quantized_results["prediction"] == quantized_results["label"]

# 両モデルの結果を結合
merged_results = pd.merge(base_results, quantized_results, on="id", suffixes=("_base", "_quantized"))

# 各モデルの正誤比較を記録
merged_results["comparison"] = merged_results.apply(
    lambda row: "both_correct" if row["is_correct_base"] and row["is_correct_quantized"]
    else "base_correct" if row["is_correct_base"]
    else "quantized_correct" if row["is_correct_quantized"]
    else "both_wrong",
    axis=1
)

# 結果をCSVに保存
merged_results.to_csv("evaluation_results.csv", index=False)

# 結果を表示
print(merged_results.head())

Evaluating Base Model...


Evaluating Base Model: 100%|██████████████| 20073/20073 [1:18:14<00:00,  4.28it/s]


Base Model evaluation completed in 4694.27 seconds.
Evaluating Quantized Model...


Evaluating Quantized Model: 100%|█████████| 20073/20073 [1:27:33<00:00,  3.82it/s]


Quantized Model evaluation completed in 5253.60 seconds.
   id                                   input_base label_base  \
0   0  二人の男性がジャンボジェット機を見ています。 2人の男性が、白い飛行機を眺めています。          2   
1   1  2人の男性が、白い飛行機を眺めています。 二人の男性がジャンボジェット機を見ています。          2   
2   2      男性が子供を抱き上げて立っています。 坊主頭の男性が子供を抱いて立っています。          2   
3   3      坊主頭の男性が子供を抱いて立っています。 男性が子供を抱き上げて立っています。          0   
4   4        草地の上で牛と男性が立っています。 農場で白い牛が男性の隣に立っています。          2   

                                     prediction_base  is_correct_base  \
0  二人の男性がジャンボジェット機を見ています。 2人の男性が、白い飛行機を眺めています。\n\...            False   
1  2人の男性が、白い飛行機を眺めています。 二人の男性がジャンボジェット機を見ています。\n\...            False   
2  男性が子供を抱き上げて立っています。 坊主頭の男性が子供を抱いて立っています。\n\n## ...            False   
3  坊主頭の男性が子供を抱いて立っています。 男性が子供を抱き上げて立っています。 男性は子供を...            False   
4  草地の上で牛と男性が立っています。 農場で白い牛が男性の隣に立っています。\n\n## 牛の...            False   

                               input_quantized label_quantized  \
0  二人の男性がジャンボジェット機を見ています。 2人の男性が、白い飛行機を眺めています。 

In [7]:
merged_results.head()

,id,input_base,label_base,prediction_base,is_correct_base,input_quantized,label_quantized,prediction_quantized,is_correct_quantized,comparison
0,0,二人の男性がジャンボジェット機を見ています。 2人の男性が、白い飛行機を眺めています。,2,二人の男性がジャンボジェット機を見ています。 2人の男性が、白い飛行機を眺めています。\n\...,False,二人の男性がジャンボジェット機を見ています。 2人の男性が、白い飛行機を眺めています。,2,二人の男性がジャンボジェット機を見ています。 2人の男性が、白い飛行機を眺めています。\n\...,False,both_wrong
1,1,2人の男性が、白い飛行機を眺めています。 二人の男性がジャンボジェット機を見ています。,2,2人の男性が、白い飛行機を眺めています。 二人の男性がジャンボジェット機を見ています。\n\...,False,2人の男性が、白い飛行機を眺めています。 二人の男性がジャンボジェット機を見ています。,2,2人の男性が、白い飛行機を眺めています。 二人の男性がジャンボジェット機を見ています。\n\...,False,both_wrong
2,2,男性が子供を抱き上げて立っています。 坊主頭の男性が子供を抱いて立っています。,2,男性が子供を抱き上げて立っています。 坊主頭の男性が子供を抱いて立っています。\n\n## ...,False,男性が子供を抱き上げて立っています。 坊主頭の男性が子供を抱いて立っています。,2,男性が子供を抱き上げて立っています。 坊主頭の男性が子供を抱いて立っています。\n\n## ...,False,both_wrong
3,3,坊主頭の男性が子供を抱いて立っています。 男性が子供を抱き上げて立っています。,0,坊主頭の男性が子供を抱いて立っています。 男性が子供を抱き上げて立っています。 男性は子供を...,False,坊主頭の男性が子供を抱いて立っています。 男性が子供を抱き上げて立っています。,0,坊主頭の男性が子供を抱いて立っています。 男性が子供を抱き上げて立っています。 男性が子供を...,False,both_wrong
4,4,草地の上で牛と男性が立っています。 農場で白い牛が男性の隣に立っています。,2,草地の上で牛と男性が立っています。 農場で白い牛が男性の隣に立っています。\n\n## 牛の...,False,草地の上で牛と男性が立っています。 農場で白い牛が男性の隣に立っています。,2,草地の上で牛と男性が立っています。 農場で白い牛が男性の隣に立っています。\n\n## 🎨,False,both_wrong


In [6]:
# 1. 全件数
total_count = len(merged_results)

# 2. baseモデルの正解数と正答率
base_correct_count = merged_results["is_correct_base"].sum()
base_accuracy = base_correct_count / total_count

# 3. quantizeモデルの正解数と正答率
quantized_correct_count = merged_results["is_correct_quantized"].sum()
quantized_accuracy = quantized_correct_count / total_count

# 4. (baseモデルで正解 ∧ quantizeで不正解) / (baseモデルで正解の件数)
base_correct_quantized_wrong_count = merged_results[
    (merged_results["is_correct_base"] == True) & (merged_results["is_correct_quantized"] == False)
].shape[0]
base_correct_quantized_wrong_ratio = base_correct_quantized_wrong_count / base_correct_count

# 結果の出力
print(f"1. 全件数: {total_count}")
print(f"2. Baseモデルの正解数: {base_correct_count} (正答率: {base_accuracy:.2%})")
print(f"3. Quantizedモデルの正解数: {quantized_correct_count} (正答率: {quantized_accuracy:.2%})")
print(f"4. (Baseモデルで正解 ∧ Quantizedモデルで不正解) / (Baseモデルで正解の件数): {base_correct_quantized_wrong_ratio:.2%}")


1. 全件数: 20073
2. Baseモデルの正解数: 0 (正答率: 0.00%)
3. Quantizedモデルの正解数: 0 (正答率: 0.00%)
4. (Baseモデルで正解 ∧ Quantizedモデルで不正解) / (Baseモデルで正解の件数): nan%


/tmp/ipykernel_2145333/2911586676.py:16: RuntimeWarning: invalid value encountered in scalar divide
  base_correct_quantized_wrong_ratio = base_correct_quantized_wrong_count / base_correct_count
